# Chapter 4

## 4.1.2

In [1]:
rdd_trans = sc.textFile('dataIn/ch04_data_transactions.txt')

In [2]:
rdd_trans.count()

1000

In [3]:
rdd_trans.take(5)

[u'2015-03-30#6:55 AM#51#68#1#9506.21',
 u'2015-03-30#7:39 PM#99#86#5#4107.59',
 u'2015-03-30#11:57 AM#79#58#7#2987.22',
 u'2015-03-30#12:46 AM#51#50#6#7501.89',
 u'2015-03-30#11:39 AM#86#24#5#8370.2']

In [4]:
rdd_trans_data = rdd_trans.map(lambda line: line.split('#'))

In [5]:
rdd_trans_data.count()

1000

In [6]:
rdd_trans_data.take(5)

[[u'2015-03-30', u'6:55 AM', u'51', u'68', u'1', u'9506.21'],
 [u'2015-03-30', u'7:39 PM', u'99', u'86', u'5', u'4107.59'],
 [u'2015-03-30', u'11:57 AM', u'79', u'58', u'7', u'2987.22'],
 [u'2015-03-30', u'12:46 AM', u'51', u'50', u'6', u'7501.89'],
 [u'2015-03-30', u'11:39 AM', u'86', u'24', u'5', u'8370.2']]

In [7]:
# create paired rdd
rdd_trans_by_cust = rdd_trans_data.map(lambda trans: (int(trans[2]), trans))

In [8]:
rdd_trans_by_cust.count()

1000

In [9]:
rdd_trans_by_cust.take(5)

[(51, [u'2015-03-30', u'6:55 AM', u'51', u'68', u'1', u'9506.21']),
 (99, [u'2015-03-30', u'7:39 PM', u'99', u'86', u'5', u'4107.59']),
 (79, [u'2015-03-30', u'11:57 AM', u'79', u'58', u'7', u'2987.22']),
 (51, [u'2015-03-30', u'12:46 AM', u'51', u'50', u'6', u'7501.89']),
 (86, [u'2015-03-30', u'11:39 AM', u'86', u'24', u'5', u'8370.2'])]

In [10]:
# Unique transactions
rdd_trans_by_cust.keys().distinct().count()

100

In [11]:
rdd_trans_by_cust.keys().distinct().take(5)

[2, 4, 6, 8, 10]

## Give a complimentary bear doll (ID 4) to the customer who made the most transactions.

In [12]:
# 1st method
(trans_count, cid) = sorted(rdd_trans_by_cust \
                            .groupByKey() \
                            .mapValues(len) \
                            .map(lambda t: (t[1], t[0])) \
                            .collect()
                           )[-1]
(trans_count, cid)

(19, 53)

In [13]:
# 2nd method
(cid, trans_count) = sorted(rdd_trans_by_cust \
                            .groupByKey() \
                            .mapValues(len) \
                            .collect(), key=lambda t: t[1], reverse=True)[0]
(cid, trans_count)

(53, 19)

In [14]:
(cid, trans_count) = sorted(rdd_trans_by_cust \
                            .countByKey() \
                            .items(), key=lambda t: t[1], reverse=True)[0]
(cid, trans_count)

(53, 19)

### Transactions made by customerId : 53

In [41]:
rdd_trans_by_cust.lookup(53)

[[u'2015-03-30', u'6:18 AM', u'53', u'42', u'5', u'2197.85'],
 [u'2015-03-30', u'4:42 AM', u'53', u'44', u'6', u'9182.08'],
 [u'2015-03-30', u'2:51 AM', u'53', u'59', u'5', u'3154.43'],
 [u'2015-03-30', u'5:57 PM', u'53', u'31', u'5', u'6649.27'],
 [u'2015-03-30', u'6:11 AM', u'53', u'33', u'10', u'2353.72'],
 [u'2015-03-30', u'9:46 PM', u'53', u'93', u'1', u'2889.03'],
 [u'2015-03-30', u'4:15 PM', u'53', u'72', u'7', u'9157.55'],
 [u'2015-03-30', u'2:42 PM', u'53', u'94', u'1', u'921.65'],
 [u'2015-03-30', u'8:30 AM', u'53', u'38', u'5', u'4000.92'],
 [u'2015-03-30', u'6:06 AM', u'53', u'12', u'6', u'2174.02'],
 [u'2015-03-30', u'3:44 AM', u'53', u'47', u'1', u'7556.32'],
 [u'2015-03-30', u'10:25 AM', u'53', u'30', u'2', u'5107.0'],
 [u'2015-03-30', u'1:48 AM', u'53', u'58', u'4', u'718.93'],
 [u'2015-03-30', u'9:31 AM', u'53', u'18', u'4', u'8214.79'],
 [u'2015-03-30', u'9:04 AM', u'53', u'68', u'4', u'9246.59'],
 [u'2015-03-30', u'1:51 AM', u'53', u'40', u'1', u'4095.5'],
 [u'2015-0

In [52]:
# Complementary Bear doll
l_l_complTrans = [["2015-03-30", "11:59 PM", "53", "4", "1", "0.00"]]

## Give a 5% discount for two or more Barbie Shopping Mall Playsets bought

#### filter() is used just to check if any customer match above criteria
#### filter() when called on a paired rdd receives a tuple (k, v)

In [15]:
rdd_trans_by_cust.filter(lambda t: int(t[1][3]) == 25 and int(t[1][4]) > 1).count()

9

In [16]:
def isMultipleBarbiShpMallPlay(trans):
    return int(trans[3]) == 25 and int(trans[4]) > 1

In [17]:
rdd_trans_by_cust.filter(lambda t: isMultipleBarbiShpMallPlay(t[1])).take(10)

[(17, [u'2015-03-30', u'6:26 PM', u'17', u'25', u'6', u'7193.11']),
 (93, [u'2015-03-30', u'7:27 AM', u'93', u'25', u'7', u'2749.15']),
 (68, [u'2015-03-30', u'1:07 AM', u'68', u'25', u'9', u'8391.61']),
 (59, [u'2015-03-30', u'1:23 AM', u'59', u'25', u'5', u'5296.69']),
 (42, [u'2015-03-30', u'9:45 AM', u'42', u'25', u'10', u'1363.97']),
 (77, [u'2015-03-30', u'10:40 PM', u'77', u'25', u'3', u'3345.81']),
 (22, [u'2015-03-30', u'12:53 PM', u'22', u'25', u'9', u'6996.42']),
 (32, [u'2015-03-30', u'12:44 AM', u'32', u'25', u'8', u'8849.5']),
 (75, [u'2015-03-30', u'5:30 PM', u'75', u'25', u'10', u'3557.01'])]

In [18]:
def apply5PercentDiscount(trans):
    if isMultipleBarbiShpMallPlay(trans):
        trans[5] = str(float(trans[5]) * 0.95)
    return trans

#### mapValues() : http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD.mapValues

In [19]:
rdd_trans_by_cust_discount = rdd_trans_by_cust.mapValues(lambda trans: apply5PercentDiscount(trans))

#### Cross check mapValues results using filter

In [20]:
rdd_trans_by_cust_discount.filter(lambda t: isMultipleBarbiShpMallPlay(t[1])).take(10)

[(17, [u'2015-03-30', u'6:26 PM', u'17', u'25', u'6', '6833.4545']),
 (93, [u'2015-03-30', u'7:27 AM', u'93', u'25', u'7', '2611.6925']),
 (68, [u'2015-03-30', u'1:07 AM', u'68', u'25', u'9', '7972.0295']),
 (59, [u'2015-03-30', u'1:23 AM', u'59', u'25', u'5', '5031.8555']),
 (42, [u'2015-03-30', u'9:45 AM', u'42', u'25', u'10', '1295.7715']),
 (77, [u'2015-03-30', u'10:40 PM', u'77', u'25', u'3', '3178.5195']),
 (22, [u'2015-03-30', u'12:53 PM', u'22', u'25', u'9', '6646.599']),
 (32, [u'2015-03-30', u'12:44 AM', u'32', u'25', u'8', '8407.025']),
 (75, [u'2015-03-30', u'5:30 PM', u'75', u'25', u'10', '3379.1595'])]

## Add a complimentary toothbrush (ID 70) to customers who bought five or more dictionaries (ID 81)

In [21]:
def isMultipleDictBought(tran):
    return int(tran[3]) == 81 and int(tran[4]) > 4

In [22]:
rdd_trans_by_cust.filter(lambda t: isMultipleDictBought(t[1])).take(10)

[(85, [u'2015-03-30', u'1:55 PM', u'85', u'81', u'7', u'9648.24']),
 (77, [u'2015-03-30', u'9:34 PM', u'77', u'81', u'5', u'7798.28']),
 (82, [u'2015-03-30', u'3:40 AM', u'82', u'81', u'6', u'3665.45']),
 (47, [u'2015-03-30', u'10:39 PM', u'47', u'81', u'10', u'1122.89']),
 (16, [u'2015-03-30', u'11:02 AM', u'16', u'81', u'9', u'9063.74']),
 (10, [u'2015-03-30', u'2:54 PM', u'10', u'81', u'10', u'9897.61'])]

In [23]:
def addToothBrush(tran):
    if isMultipleDictBought(tran):
        from copy import copy
        cloned_tran = copy(tran)
        cloned_tran[5] = "0.00"
        cloned_tran[3] = "70"
        cloned_tran[4] = "1"
        return [tran, cloned_tran]
    else:
        return [tran]

#### flatMapValues : http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD.flatMapValues

In [24]:
rdd_trans_by_cust_with_comp = rdd_trans_by_cust.flatMapValues(lambda tran: addToothBrush(tran))

In [25]:
rdd_trans_by_cust_with_comp.count()

1006

In [26]:
rdd_trans_by_cust_with_comp.filter(lambda t: isMultipleDictBought(t[1])).take(10)

[(85, [u'2015-03-30', u'1:55 PM', u'85', u'81', u'7', u'9648.24']),
 (77, [u'2015-03-30', u'9:34 PM', u'77', u'81', u'5', u'7798.28']),
 (82, [u'2015-03-30', u'3:40 AM', u'82', u'81', u'6', u'3665.45']),
 (47, [u'2015-03-30', u'10:39 PM', u'47', u'81', u'10', u'1122.89']),
 (16, [u'2015-03-30', u'11:02 AM', u'16', u'81', u'9', u'9063.74']),
 (10, [u'2015-03-30', u'2:54 PM', u'10', u'81', u'10', u'9897.61'])]

In [27]:
def isCompToothBrush(tran):
    return int(tran[3]) == 70 and float(tran[5]) == 0.0

In [28]:
rdd_trans_by_cust_with_comp.filter(lambda t: isCompToothBrush(t[1])).take(10)

[(85, [u'2015-03-30', u'1:55 PM', u'85', '70', '1', '0.00']),
 (77, [u'2015-03-30', u'9:34 PM', u'77', '70', '1', '0.00']),
 (82, [u'2015-03-30', u'3:40 AM', u'82', '70', '1', '0.00']),
 (47, [u'2015-03-30', u'10:39 PM', u'47', '70', '1', '0.00']),
 (16, [u'2015-03-30', u'11:02 AM', u'16', '70', '1', '0.00']),
 (10, [u'2015-03-30', u'2:54 PM', u'10', '70', '1', '0.00'])]

## Give a pair of pajamas (ID 63) to the customer who spent the most over all

In [32]:
# First convert transaction amount to float
rdd_amount_by_cust = rdd_trans_by_cust_with_comp.mapValues(lambda tran: float(tran[5]))

In [33]:
rdd_amount_by_cust.take(5)

[(51, 9506.21), (99, 4107.59), (79, 2987.22), (51, 7501.89), (86, 8370.2)]

### [foldByKey](http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD.foldByKey)

In [34]:
l_total = rdd_amount_by_cust.foldByKey(0, lambda v1, v2: v1 + v2).collect()

In [35]:
len(l_total)

100

In [37]:
(cid, total) = sorted(l_total, key=lambda t: t[1], reverse=True)[0]

In [38]:
(cid, total)

(76, 100049.0)

In [46]:
l_total1 = rdd_amount_by_cust.reduceByKey(lambda v1, v2: v1 + v2).collect()
from operator import add
l_total1 = rdd_amount_by_cust.reduceByKey(add).collect()

In [47]:
len(l_total1)

100

In [49]:
(cid, total) = sorted(l_total1, key=lambda t: t[1], reverse=True)[0]

In [50]:
(cid, total)

(76, 100049.0)

In [53]:
# Complementary pajamas
l_l_complTrans += [["2015-03-30", "11:59 PM", "76", "63", "1", "0.00"]]

### Add complementary transactions to original rdd and save it to a file
### Comp trans could have been added using foldByKey() as done above but this is another method

In [54]:
l_l_complTrans

[['2015-03-30', '11:59 PM', '53', '4', '1', '0.00'],
 ['2015-03-30', '11:59 PM', '76', '63', '1', '0.00']]

In [55]:
# Create rdd from comp trans
rdd_comp_trans = sc.parallelize(l_l_complTrans) \
                    .map(lambda tran: (int(tran[2]), tran))

In [56]:
rdd_comp_trans.take(5)

[(53, ['2015-03-30', '11:59 PM', '53', '4', '1', '0.00']),
 (76, ['2015-03-30', '11:59 PM', '76', '63', '1', '0.00'])]

In [57]:
rdd_final_trans_by_cust = rdd_trans_by_cust_with_comp.union(rdd_comp_trans)

In [58]:
rdd_final_trans_by_cust.count()

1008

In [60]:
# Save the rdd
rdd_final_trans_by_cust.map(lambda t: '#'.join(t[1])).saveAsTextFile('dataOut/ch04_finaltransbycust')

## List of all products purchased by each customer
### [aggregateByKey](http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD.aggregateByKey)

In [64]:
rdd_prodId = rdd_trans_by_cust.aggregateByKey([],
                                              lambda l_prodId, tran: l_prodId + [tran[3]],
                                              lambda l_prodId1, l_prodId2: l_prodId1 + l_prodId2,
                                              2
                                             )

In [65]:
rdd_prodId.count()

100

In [66]:
rdd_prodId.take(5)

[(2,
  [u'5',
   u'49',
   u'64',
   u'48',
   u'1',
   u'53',
   u'67',
   u'48',
   u'6',
   u'65',
   u'31',
   u'15',
   u'60',
   u'79',
   u'74']),
 (4,
  [u'16', u'86', u'19', u'49', u'99', u'2', u'94', u'35', u'4', u'10', u'7']),
 (6, [u'90', u'47', u'38', u'15', u'55', u'28', u'51']),
 (8, [u'35', u'93', u'45', u'92', u'65', u'65', u'93', u'84', u'44', u'22']),
 (10, [u'1', u'36', u'83', u'17', u'46', u'30', u'81'])]

## 4.2
### rdd partition count
#### http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD.getNumPartitions

In [75]:
rdd_trans_by_cust.getNumPartitions()

2

## 4.2.2

In [90]:
rdd1 = sc.parallelize(range(10000))
rdd1.getNumPartitions()

8

In [92]:
# This won't cause a shuffle
rdd1.map(lambda x: (x, x*x)).map(lambda (k, v): (v, k)).take(10)

[(0, 0),
 (1, 1),
 (4, 2),
 (9, 3),
 (16, 4),
 (25, 5),
 (36, 6),
 (49, 7),
 (64, 8),
 (81, 9)]

In [93]:
# reduceByKey() causes shuffle
rdd1.map(lambda x: (x, x*x)).reduceByKey(lambda (v1, v2): v1 + v2).take(10)

[(0, 0),
 (2048, 4194304),
 (9288, 86266944),
 (5592, 31270464),
 (9752, 95101504),
 (8, 64),
 (9816, 96353856),
 (8320, 69222400),
 (7024, 49336576),
 (6888, 47444544)]

## 4.2.4

In [94]:
import random
l = [random.randrange(100) for x in range(500)]

In [95]:
rdd2 = sc.parallelize(l, 30)

In [96]:
rdd2.take(10)

[88, 24, 95, 49, 65, 12, 6, 37, 50, 38]

In [97]:
rdd2.getNumPartitions()

30

#### glom (the word means to grab) gathers elements of each partition into an array and returns a new RDD with those arrays as elements. The number of elements in the new RDD is equal to the number of its partitions. The partitioner is removed in the process.

In [98]:
rdd3 = rdd2.glom()

In [99]:
rdd3.take(10)

[[88, 24, 95, 49, 65, 12, 6, 37, 50, 38, 68, 66, 77, 27, 69, 3],
 [31, 8, 55, 1, 7, 63, 85, 13, 55, 45, 20, 59, 94, 41, 91, 44],
 [79, 75, 18, 17, 12, 80, 97, 80, 21, 99, 91, 73, 17, 16, 63, 2],
 [98, 94, 9, 16, 14, 42, 73, 10, 98, 77, 90, 90, 6, 88, 99, 25],
 [68, 19, 48, 85, 48, 40, 35, 79, 65, 67, 65, 7, 94, 74, 19, 65],
 [20, 51, 49, 26, 64, 55, 6, 70, 37, 3, 2, 49, 81, 8, 14, 43],
 [99, 31, 82, 70, 55, 1, 28, 39, 92, 69, 11, 93, 44, 94, 69, 13],
 [22, 13, 88, 95, 39, 36, 22, 29, 72, 76, 79, 45, 19, 92, 49, 2],
 [81, 86, 38, 14, 2, 2, 90, 95, 67, 79, 61, 80, 75, 39, 41, 23],
 [64, 81, 72, 46, 76, 99, 76, 36, 91, 52, 76, 72, 18, 56, 57, 66]]

In [100]:
rdd3.count()

30

## 4.3.1 Joining data

### names of products with totals sold, sorted alphabetically

In [101]:
rdd_trans_data.take(5)

[[u'2015-03-30', u'6:55 AM', u'51', u'68', u'1', u'9506.21'],
 [u'2015-03-30', u'7:39 PM', u'99', u'86', u'5', u'4107.59'],
 [u'2015-03-30', u'11:57 AM', u'79', u'58', u'7', u'2987.22'],
 [u'2015-03-30', u'12:46 AM', u'51', u'50', u'6', u'7501.89'],
 [u'2015-03-30', u'11:39 AM', u'86', u'24', u'5', u'8370.2']]

In [104]:
rdd_prod_by_trans = rdd_trans_data.map(lambda trans: (int(trans[3]), trans))

In [109]:
rdd_prod_by_trans.take(5)

[(68, [u'2015-03-30', u'6:55 AM', u'51', u'68', u'1', u'9506.21']),
 (86, [u'2015-03-30', u'7:39 PM', u'99', u'86', u'5', u'4107.59']),
 (58, [u'2015-03-30', u'11:57 AM', u'79', u'58', u'7', u'2987.22']),
 (50, [u'2015-03-30', u'12:46 AM', u'51', u'50', u'6', u'7501.89']),
 (24, [u'2015-03-30', u'11:39 AM', u'86', u'24', u'5', u'8370.2'])]

In [122]:
from operator import add
rdd_totals_per_prod = rdd_prod_by_trans \
                        .mapValues(lambda trans: float(trans[5])) \
                        .reduceByKey(add)

rdd_totals_per_prod.take(5)

[(2, 27613.84),
 (4, 63520.21000000001),
 (6, 43252.97),
 (8, 43761.869999999995),
 (10, 47085.59)]

In [125]:
rdd_prodid_by_prod = sc.textFile('dataIn/ch04_data_products.txt').map(lambda p: p.split('#')).map(lambda prod: (int(prod[0]), prod))
rdd_prodid_by_prod.take(5)

[(1,
  [u'1', u'ROBITUSSIN PEAK COLD NIGHTTIME COLD PLUS FLU', u'9721.89', u'10']),
 (2, [u'2', u'Mattel Little Mommy Doctor Doll', u'6060.78', u'6']),
 (3, [u'3', u'Cute baby doll, battery', u'1808.79', u'2']),
 (4, [u'4', u'Bear doll', u'51.06', u'6']),
 (5, [u'5', u'LEGO Legends of Chima', u'849.36', u'6'])]

#### Join : http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD.join

In [126]:
rdd_prodid_by_prod_totals = rdd_totals_per_prod.join(rdd_prodid_by_prod)
rdd_prodid_by_prod_totals.take(5)

[(4, (63520.21000000001, [u'4', u'Bear doll', u'51.06', u'6'])),
 (8, (43761.869999999995, [u'8', u'LEGO Star Wars', u'7592.44', u'4'])),
 (12, (45396.11, [u'12', u'LEGO Hero Factory', u'6911.2', u'1'])),
 (16, (46664.43000000001, [u'16', u'LEGO Classic', u'9933.3', u'10'])),
 (24, (46973.67, [u'24', u'LEGO Pirates', u'4150.34', u'2']))]

In [129]:
rdd_prod_name_by_totals_sold = rdd_prodid_by_prod_totals \
                                .mapValues(lambda prod: (prod[1][1], prod[0])) \
                                .map(lambda prod: (prod[1][0], prod[1][1]))
rdd_prod_name_by_totals_sold.take(5)

[(u'Bear doll', 63520.21000000001),
 (u'LEGO Star Wars', 43761.869999999995),
 (u'LEGO Hero Factory', 45396.11),
 (u'LEGO Classic', 46664.43000000001),
 (u'LEGO Pirates', 46973.67)]

In [130]:
sorted(rdd_prod_name_by_totals_sold.collect(), key=lambda p: p[0])

[(u'AMBROSIA TRIFIDA POLLEN', 48601.89),
 (u'ATOPALM MUSCLE AND JOINT', 31049.07),
 (u'Acyclovir', 26047.72),
 (u'Alphanate', 50917.700000000004),
 (u'Ativan', 31498.84),
 (u'Barbie Beach Ken Doll', 60695.88),
 (u'Barbie Bedtime Princess Doll', 38850.68),
 (u'Barbie Shopping Mall Playset', 60353.25),
 (u'Bear doll', 63520.21000000001),
 (u'Brimonidine Tartrate', 59283.19),
 (u'Bronchitis', 50433.88),
 (u'CUPRUM METALLICUM', 34805.73),
 (u'CVS', 32156.14),
 (u'Caldyphen Clear', 56677.71000000001),
 (u'Childrens Non-Drowsy', 16470.08),
 (u'Cosopt', 43765.64),
 (u'Cyanocobalamin', 75192.53),
 (u'Dictionary', 66420.93),
 (u'Disposable diapers', 68147.48000000001),
 (u'Essentials Crash Tag Team Racing PSP', 57708.119999999995),
 (u'Essentials Dantes Inferno PSP', 37302.96),
 (u'Essentials Medal of Honor: Heroes 2 PSP', 20736.739999999998),
 (u'Essentials Tekken 6 PS3', 57313.90000000001),
 (u'Far Cry 4 Limited Edition for Xbox One', 82055.45999999999),
 (u'GAM X360  Dead Space 3 Limited Edi

In [131]:
sorted(rdd_prod_name_by_totals_sold.collect(), key=lambda p: p[1], reverse=True)[0]

(u'Notebook Lenovo U330p, 59-390439', 186539.8)

In [165]:
rdd_prod_name_by_totals_sold.top(1, key=lambda p: p[1])

[(u'Notebook Lenovo U330p, 59-390439', 186539.8)]

### a list of products the company didn’t sell yesterday

#### 1. use leftOurJoin or rightOuterJoin

In [136]:
rdd_products_not_sold = rdd_prodid_by_prod \
                            .leftOuterJoin(rdd_totals_per_prod) \
                            .filter(lambda p: p[1][1] is None) \
                            .mapValues(lambda prod: prod[0][1])
rdd_products_not_sold.take(5)

[(20, u'LEGO Elves'),
 (3, u'Cute baby doll, battery'),
 (43, u'Tomb Raider PC'),
 (63, u'Pajamas')]

#### 2. use subtractByKey

In [140]:
rdd_products_not_sold = rdd_prodid_by_prod \
                            .subtractByKey(rdd_totals_per_prod) \
                            .mapValues(lambda prod: prod[1])
rdd_products_not_sold.take(5)

[(20, u'LEGO Elves'),
 (3, u'Cute baby doll, battery'),
 (43, u'Tomb Raider PC'),
 (63, u'Pajamas')]

#### 3. use cogroup

In [149]:
rdd_products_not_sold = rdd_totals_per_prod \
                            .cogroup(rdd_prodid_by_prod) \
                            .filter(lambda p: len(p[1][0].data) == 0) \
                            .mapValues(lambda itrs: itrs[1].data[0][1])
rdd_products_not_sold.take(5)

[(20, u'LEGO Elves'),
 (3, u'Cute baby doll, battery'),
 (43, u'Tomb Raider PC'),
 (63, u'Pajamas')]

#### intersection

In [152]:
rdd1 = sc.parallelize([1, 1, 2, 3, 4, 5])
rdd2 = sc.parallelize([1, 2, 2, 3, 6])
rdd1.intersection(rdd2).collect()

[1, 2, 3]

In [153]:
rdd2.intersection(rdd1).collect()

[1, 2, 3]

#### cartesian

In [154]:
rdd1 = sc.parallelize([1, 1, 2, 3,])
rdd2 = sc.parallelize([1, 2, 2, 3, 6])
rdd1.cartesian(rdd2).collect()

[(1, 1),
 (1, 2),
 (1, 2),
 (1, 3),
 (1, 6),
 (1, 1),
 (1, 2),
 (1, 2),
 (1, 3),
 (1, 6),
 (2, 1),
 (2, 2),
 (2, 2),
 (2, 3),
 (2, 6),
 (3, 1),
 (3, 2),
 (3, 2),
 (3, 3),
 (3, 6)]

#### zip

In [158]:
rdd1 = sc.parallelize([1,2,3])
rdd2 = sc.parallelize(["n4","n5","n6"])
rdd1.zip(rdd2).collect()

[(1, 'n4'), (2, 'n5'), (3, 'n6')]

### 4.3.2 Sorting

#### http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD.sortBy
#### http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD.sortByKey

In [156]:
rdd_prod_name_by_totals_sold.sortBy(lambda t: t[0]).collect()

[(u'AMBROSIA TRIFIDA POLLEN', 48601.89),
 (u'ATOPALM MUSCLE AND JOINT', 31049.07),
 (u'Acyclovir', 26047.72),
 (u'Alphanate', 50917.700000000004),
 (u'Ativan', 31498.84),
 (u'Barbie Beach Ken Doll', 60695.88),
 (u'Barbie Bedtime Princess Doll', 38850.68),
 (u'Barbie Shopping Mall Playset', 60353.25),
 (u'Bear doll', 63520.21000000001),
 (u'Brimonidine Tartrate', 59283.19),
 (u'Bronchitis', 50433.88),
 (u'CUPRUM METALLICUM', 34805.73),
 (u'CVS', 32156.14),
 (u'Caldyphen Clear', 56677.71000000001),
 (u'Childrens Non-Drowsy', 16470.08),
 (u'Cosopt', 43765.64),
 (u'Cyanocobalamin', 75192.53),
 (u'Dictionary', 66420.93),
 (u'Disposable diapers', 68147.48000000001),
 (u'Essentials Crash Tag Team Racing PSP', 57708.119999999995),
 (u'Essentials Dantes Inferno PSP', 37302.96),
 (u'Essentials Medal of Honor: Heroes 2 PSP', 20736.739999999998),
 (u'Essentials Tekken 6 PS3', 57313.90000000001),
 (u'Far Cry 4 Limited Edition for Xbox One', 82055.45999999999),
 (u'GAM X360  Dead Space 3 Limited Edi

In [157]:
rdd_prod_name_by_totals_sold.sortByKey().collect()

[(u'AMBROSIA TRIFIDA POLLEN', 48601.89),
 (u'ATOPALM MUSCLE AND JOINT', 31049.07),
 (u'Acyclovir', 26047.72),
 (u'Alphanate', 50917.700000000004),
 (u'Ativan', 31498.84),
 (u'Barbie Beach Ken Doll', 60695.88),
 (u'Barbie Bedtime Princess Doll', 38850.68),
 (u'Barbie Shopping Mall Playset', 60353.25),
 (u'Bear doll', 63520.21000000001),
 (u'Brimonidine Tartrate', 59283.19),
 (u'Bronchitis', 50433.88),
 (u'CUPRUM METALLICUM', 34805.73),
 (u'CVS', 32156.14),
 (u'Caldyphen Clear', 56677.71000000001),
 (u'Childrens Non-Drowsy', 16470.08),
 (u'Cosopt', 43765.64),
 (u'Cyanocobalamin', 75192.53),
 (u'Dictionary', 66420.93),
 (u'Disposable diapers', 68147.48000000001),
 (u'Essentials Crash Tag Team Racing PSP', 57708.119999999995),
 (u'Essentials Dantes Inferno PSP', 37302.96),
 (u'Essentials Medal of Honor: Heroes 2 PSP', 20736.739999999998),
 (u'Essentials Tekken 6 PS3', 57313.90000000001),
 (u'Far Cry 4 Limited Edition for Xbox One', 82055.45999999999),
 (u'GAM X360  Dead Space 3 Limited Edi

#### takeOrdered
top and takeOrdered don’t perform full sorting of data among partitions. Instead, they first take the first (or
last) n elements from each partition, merge the results, and then take the first (or last) n elements from the merged list. This is much faster than doing sortBy and then calling take, because much less data needs to be
transferred over the network.

In [159]:
rdd1 = sc.parallelize([8, 3, 4, 1, 4, 6, 5, 0, 9])

In [160]:
rdd1.takeOrdered(5)

[0, 1, 3, 4, 4]

In [162]:
rdd1.takeOrdered(10, key=lambda x: -x)

[9, 8, 6, 5, 4, 4, 3, 1, 0]

In [163]:
rdd1.top(2)

[9, 8]

In [164]:
rdd1.top(2, key=lambda x: -x)

[0, 1]

### 4.3.3 calculate the average, maximum, minimum, and total price of products bought per customer

In [166]:
rdd_trans_by_cust.take(5)

[(51, [u'2015-03-30', u'6:55 AM', u'51', u'68', u'1', u'9506.21']),
 (99, [u'2015-03-30', u'7:39 PM', u'99', u'86', u'5', u'4107.59']),
 (79, [u'2015-03-30', u'11:57 AM', u'79', u'58', u'7', u'2987.22']),
 (51, [u'2015-03-30', u'12:46 AM', u'51', u'50', u'6', u'7501.89']),
 (86, [u'2015-03-30', u'11:39 AM', u'86', u'24', u'5', u'8370.2'])]

In [167]:
def createCombiner(t):
    totalSpent = float(t[5])
    quantity = int(t[4])
    pricePerProduct = totalSpent / quantity
    # min, max, quantity, total
    return (pricePerProduct, pricePerProduct, quantity, totalSpent)

def mergeValue((mini, maxi, qt, total), t):
    (mini_t, maxi_t, qt_t, total_t) = createCombiner(t)
    return (min(mini, mini_t), max(maxi, maxi_t), qt + qt_t, total + total_t)

def mergeCombiners((mini1, maxi1, qt1, total1), (mini2, maxi2, qt2, total2)):
    return (min(mini1, mini2), max(maxi1, maxi2), qt1 + qt2, total1 + total2)

In [168]:
rdd_cust_stats = rdd_trans_by_cust.combineByKey(createCombiner, mergeValue, mergeCombiners)

In [169]:
rdd_cust_stats.take(5)

[(2, (86.128, 2011.855, 93, 70601.0)),
 (4, (63.18222222222222, 6614.49, 58, 41801.35)),
 (6, (100.167, 3616.185, 35, 30549.28)),
 (8, (5.862, 2333.436666666667, 66, 38603.81)),
 (10, (635.6588888888889, 3540.085, 41, 46120.25))]

In [170]:
rdd_cust_stats.count()

100

In [171]:
# Now, find avg for each customer
rdd_avg_by_cust = rdd_cust_stats.mapValues(lambda (mini, maxi, qt, total): (mini, maxi, qt, total, total/qt))
rdd_avg_by_cust.take(5)

[(2, (86.128, 2011.855, 93, 70601.0, 759.1505376344086)),
 (4, (63.18222222222222, 6614.49, 58, 41801.35, 720.7129310344827)),
 (6, (100.167, 3616.185, 35, 30549.28, 872.8365714285713)),
 (8, (5.862, 2333.436666666667, 66, 38603.81, 584.9062121212121)),
 (10, (635.6588888888889, 3540.085, 41, 46120.25, 1124.8841463414635))]

In [173]:
# Save to file
rdd_avg_by_cust \
    .map(lambda (custid, (mini, maxi, qt, total, avg)): '{:d}#{:.2f}#{:.2f}#{:d}#{:.2f}#{:.2f}'.format(custid, mini, maxi, qt, total, avg)) \
    .saveAsTextFile('dataOut/ch04_avgbycust')

In [ ]:
### 4.4.1

In [20]:
import random
l = [random.randrange(10) for x in range(500)]
rdd_list = sc.parallelize(l, 5)

In [21]:
rdd_list.take(5)

[5, 1, 7, 6, 5]

In [22]:
rdd_pair = rdd_list.map(lambda x: (x, x*x))
rdd_pair.take(5)

[(5, 25), (1, 1), (7, 49), (6, 36), (5, 25)]

In [23]:
print(rdd_pair.toDebugString())

(5) PythonRDD[21] at RDD at PythonRDD.scala:48 []
 |  ParallelCollectionRDD[18] at parallelize at PythonRDD.scala:480 []


In [24]:
rdd_reduced = rdd_pair.reduceByKey(lambda v1, v2: v1 + v2)
rdd_reduced.take(5)

[(0, 0), (5, 1150), (1, 47), (6, 1836), (2, 196)]

In [25]:
print(rdd_reduced.toDebugString())

(5) PythonRDD[28] at RDD at PythonRDD.scala:48 []
 |  MapPartitionsRDD[25] at mapPartitions at PythonRDD.scala:427 []
 |  ShuffledRDD[24] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(5) PairwiseRDD[23] at reduceByKey at <ipython-input-24-678881a2837a>:1 []
    |  PythonRDD[22] at reduceByKey at <ipython-input-24-678881a2837a>:1 []
    |  ParallelCollectionRDD[18] at parallelize at PythonRDD.scala:480 []


In [26]:
rdd_final = rdd_reduced.mapPartitions(lambda itr: ["K="+str(k)+",V="+str(v) for (k,v) in itr])
rdd_final.collect()

['K=0,V=0',
 'K=5,V=1150',
 'K=1,V=47',
 'K=6,V=1836',
 'K=2,V=196',
 'K=7,V=2401',
 'K=8,V=3328',
 'K=3,V=549',
 'K=9,V=4212',
 'K=4,V=736']

In [27]:
print(rdd_final.toDebugString())

(5) PythonRDD[29] at collect at <ipython-input-26-cdfcfe487515>:2 []
 |  MapPartitionsRDD[25] at mapPartitions at PythonRDD.scala:427 []
 |  ShuffledRDD[24] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(5) PairwiseRDD[23] at reduceByKey at <ipython-input-24-678881a2837a>:1 []
    |  PythonRDD[22] at reduceByKey at <ipython-input-24-678881a2837a>:1 []
    |  ParallelCollectionRDD[18] at parallelize at PythonRDD.scala:480 []


### 4.4.3

In [50]:
sc.setCheckpointDir('dataOut/')

In [48]:
import random
l = [random.randrange(10) for x in range(500)]

In [63]:
rdd_reduced = sc.parallelize(l, 5).map(lambda x: (x, x*x)).reduceByKey(lambda v1, v2: v1+v2)

In [64]:
# checkpoint must be called before any jobs are executed on the RDD
# and the RDD has to be materialized afterward (some action has to be called on it)
# for the checkpointing to be done.
rdd_reduced.checkpoint()

In [65]:
print(rdd_reduced.toDebugString())

(5) PythonRDD[46] at RDD at PythonRDD.scala:48 []
 |  MapPartitionsRDD[45] at mapPartitions at PythonRDD.scala:427 []
 |  ShuffledRDD[44] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(5) PairwiseRDD[43] at reduceByKey at <ipython-input-63-b27adda5fbb8>:3 []
    |  PythonRDD[42] at reduceByKey at <ipython-input-63-b27adda5fbb8>:3 []
    |  ParallelCollectionRDD[41] at parallelize at PythonRDD.scala:480 []


In [66]:
# Checkpoint was called before any job (actions) was called on rdd
# With this action, rdd has materialized.
rdd_reduced.count()

10

In [67]:
print(rdd_reduced.toDebugString())

(5) PythonRDD[46] at RDD at PythonRDD.scala:48 []
 |  ReliableCheckpointRDD[48] at count at <ipython-input-66-5458b21af941>:1 []


In [68]:
rdd_reduced.getCheckpointFile()

u'file:/home/deepakt/Documents/dev/spark/sia/sia-ch4/dataOut/a67dddb7-4e8e-45f9-9456-0e0227bf5a5d/rdd-46'

In [69]:
rdd_reduced.isCheckpointed()

True

In [70]:
rdd_reduced.isLocallyCheckpointed()

False

### 4.5.1

#### accumulators in Python cannot be named

In [6]:
accum = sc.accumulator(0)

In [7]:
rdd_l = sc.parallelize(range(1000000))

In [8]:
rdd_l.foreach(lambda x: accum.add(x))

In [9]:
accum.value

499999500000

### 4.5.2 Broadcast variables

In [13]:
import random
l = [random.randrange(10) for x in range(10)]

In [14]:
l_broadcast = sc.broadcast(l)

In [15]:
l_broadcast.value

[0, 5, 5, 7, 0, 7, 5, 4, 3, 0]

In [20]:
l_rdd = sc.parallelize([random.randrange(10) for x in range(100)]).map(lambda x: x+l_broadcast.value[random.randrange(10)])

In [21]:
l_rdd.take(5)

[7, 5, 13, 9, 4]